In [1]:
!nvidia-smi

Tue Aug  2 16:47:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 30%   45C    P3    78W / 350W |   4595MiB / 24576MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
world.reset()

NameError: name 'world' is not defined

In [5]:
import getpass
user = getpass.getuser()

In [6]:
from omni.isaac.kit import SimulationApp

usd_path = f"omniverse://localhost/Users/{user}/test_allegro.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path, "livesync_usd": usd_path})


Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/apps', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-bfdf7cfc-010c-4416-befe-cf7f56fd2b38.json

OmniAssetFileFormat
2022-08-02 23:47:45 [2,132ms] [Error] [rtx.neuraylib.plugin] [DYNLIB:IO]   0.1   DYNLIB io   error: /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so: cannot open shared object file: No such file or directory
2022-08-02 23:47:45 [2,132ms] [Error] [rtx.neuraylib.plugin] [INDEX:MAIN]   0.1   INDEX  main error: Failed to load /home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.0/kit/python/bin/libs/iray/libnvindex.so


[1.769s] [ext: omni.mdl-0.1.0] startup
[1.789s] [ext: omni.kit.menu.create-1.0.2] startup
[1.790s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.791s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.791s] [ext: omni.kit.material.library-1.3.10] startup
[1.794s] [ext: omni.kit.window.property-1.6.3] startup
[1.795s] [ext: omni.kit.context_menu-1.3.9] startup
[1.799s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.799s] [ext: omni.kit.stage_templates-1.1.2] startup
[1.801s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.804s] [ext: omni.kit.window.file-1.3.16] startup
[1.806s] [ext: omni.debugdraw-0.1.0] startup
[1.812s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.824s] [ext: omni.kit.widget.prompt-1.0.1] startup
[1.825s] [ext: omni.kit.property.usd-3.14.8] startup
[1.873s] [ext: omni.hydra.engine.stats-1.0.0] startup
[1.885s] [ext: omni.kit.widget.settings-1.0.0] startup
[1.888s] [ext: omni.graph.tools-1.3.5] startup
[1.941s] [ext: omni.graph.core-2.27.0

2022-08-02 23:47:50 [6,640ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_2/visuals/base_side_panel_19 has corrupted data in primvar st: buffer size 6 doesn't match expected size 72 in faceVarying primvars
2022-08-02 23:47:50 [6,640ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_2/visuals/base_side_panel_19 has corrupted data in primvar st_1: buffer size 30 doesn't match expected size 72 in faceVarying primvars
2022-08-02 23:47:50 [6,640ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_2/visuals/base_side_panel_20 has corrupted data in primvar st: buffer size 18 doesn't match expected size 72 in faceVarying primvars
2022-08-02 23:47:50 [6,640ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_2/visuals/base_side_panel_20 has corrupted data in primvar st_1: buffer size 18 doesn't match expected size 72 in faceVarying primvars
2022-08-02 23:47:50 [6,640ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_0/visuals/cabinet_door_surface_7 has corrupte

2022-08-02 23:47:54 [10,881ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_0/visuals/handle_6 has corrupted data in primvar st: buffer size 1962 doesn't match expected size 4008 in faceVarying primvars
2022-08-02 23:47:54 [10,881ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_1/visuals/drawer_front_8 has corrupted data in primvar st: buffer size 24 doesn't match expected size 48 in faceVarying primvars
2022-08-02 23:47:54 [10,881ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_1/visuals/drawer_side_9 has corrupted data in primvar st: buffer size 36 doesn't match expected size 72 in faceVarying primvars
2022-08-02 23:47:54 [10,881ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_1/visuals/drawer_back_11 has corrupted data in primvar st: buffer size 24 doesn't match expected size 48 in faceVarying primvars
2022-08-02 23:47:54 [10,881ms] [Warning] [omni.hydra] Mesh /World/Game/mobility/link_1/visuals/drawer_side_10 has corrupted data in primvar st: buf

In [7]:
from omni.isaac.core import World
world = World()

In [8]:
from hand_env import HandEnv

# env = HandEnv("/World/Hand/Bones/l_carpal_mid", "/World/Hand*/Bones/l_thumbSkeleton_grp/l_distalThumb_mid")
env = HandEnv("/World/allegro*/allegro_mount", "/World/AnchorXform")

In [9]:
world.reset()

In [11]:
env.start()
world.scene.add(env.robots)

simlation context numpy None
num_envs 1


In [35]:
env.xforms.set_world_poses(positions=np.array([[0,0,0]]), orientations = np.array([[1, 0, 0, 0]])) # WXYZ

for _ in range(30):
    world.step(render=True)

In [16]:
hand_pos, hand_rot = env.robots.get_world_poses()
hand_pos, hand_rot

(array([[-0.01107822,  0.03002543,  0.04911713]], dtype=float32),
 array([[0.27375883, 0.1561453 , 0.1577546 , 0.93583554]], dtype=float32))

In [17]:
env.robots.get_joint_positions()

array([[-0.13209808, -0.10292442, -0.08314592,  0.2735379 , -0.05409821,
        -0.27030557, -0.3550904 , -0.01357761, -0.02220257, -0.315826  ,
        -0.32352686,  1.1855806 , -0.01220552, -0.02537572, -0.0118597 ,
         1.762732  ]], dtype=float32)

In [18]:
for i, (path, limit) in enumerate(zip(env.robots._physics_view.dof_paths[0], env.robots.get_dof_limits()[0])):
    print(i, path, limit)

0 /World/allegro/palm_link/index_joint_0 [-0.5584889  0.5584889]
1 /World/allegro/palm_link/middle_joint_0 [-0.5584889  0.5584889]
2 /World/allegro/palm_link/ring_joint_0 [-0.5584889  0.5584889]
3 /World/allegro/palm_link/thumb_joint_0 [0.27924445 1.5707499 ]
4 /World/allegro/index_link_0/index_joint_1 [-0.27924445  1.7278249 ]
5 /World/allegro/middle_link_0/middle_joint_1 [-0.27924445  1.7278249 ]
6 /World/allegro/ring_link_0/ring_joint_1 [-0.27924445  1.7278249 ]
7 /World/allegro/thumb_link_0/thumb_joint_1 [-0.33160278  1.1518834 ]
8 /World/allegro/index_link_1/index_joint_2 [-0.27924445  1.7278249 ]
9 /World/allegro/middle_link_1/middle_joint_2 [-0.27924445  1.7278249 ]
10 /World/allegro/ring_link_1/ring_joint_2 [-0.27924445  1.7278249 ]
11 /World/allegro/thumb_link_1/thumb_joint_2 [-0.27924445  1.7278249 ]
12 /World/allegro/index_link_2/index_joint_3 [-0.27924445  1.7278249 ]
13 /World/allegro/middle_link_2/middle_joint_3 [-0.27924445  1.7278249 ]
14 /World/allegro/ring_link_2/ring

# World Pose

In [ ]:
target_pos, target_rot = (np.array([[0.2, 0., 0.]], dtype=np.float32), np.array([[1., 0., 0., 0.]], dtype=np.float32))

In [ ]:
env.robots.set_world_poses(target_pos, target_rot)

In [ ]:
world.step(render=True)

In [ ]:
world.reset()

# Stage one: move to the correct place

In [19]:
from utils import get_mesh_bboxes
bboxes_list = get_mesh_bboxes("handle_6") 

In [20]:
bboxes_list

[[Gf.Vec3d(0.5999997598286658, 0.16887575486565293, 1.0008063018276903),
  Gf.Vec3d(0.6388455895036502, 0.21179346534321167, 1.043723318071727)]]

In [21]:
# get center and min x axis
min_x = bboxes_list[0][0][0] # 
center_list = [(e[1] + e[0]) / 2 for e in bboxes_list] # box center

In [22]:
grasp_list = [[min_x - 0.1, c[1], c[2] - 0.12] for c in center_list]

In [23]:
grasp_list

[[0.49999975982866585, 0.1903346101044323, 0.9022648099497085]]

In [24]:
graps_pos = np.array(grasp_list, dtype=np.float32)

base_rotation = [0.38268, 0, 0, 0.92388]
grasp_rot = np.array([base_rotation], dtype=np.float32)# XYZW
     

In [25]:
graps_pos, grasp_rot

(array([[0.49999976, 0.1903346 , 0.90226483]], dtype=float32),
 array([[0.38268, 0.     , 0.     , 0.92388]], dtype=float32))

In [1]:
world.reset()

NameError: name 'world' is not defined

In [27]:
env.xforms.set_world_poses(graps_pos, grasp_rot)

In [ ]:
# env.robots.set_world_poses(graps_pos, grasp_rot)


In [28]:
for _ in range(500):
    world.step(render=True)

# Finger Pose

In [29]:
tt = 120

In [30]:
# pos = np.array([
#     [
#      0, 0, 0, np.pi/2, 
#      0,0,0,0,
#      0,0,0,0,
#      0,0,0,0,   
#     ],
# ])

# for i in range(tt):
#     env.robots.set_joint_position_targets(pos * i / tt) # 
#     world.step(render=True)
    
finger_pos = np.array([
    [
     0, 0, 0, np.pi/2 + np.pi/18, 
     np.pi/5, np.pi/5, np.pi/5, 0,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
     np.pi/5, np.pi/5, np.pi/5, np.pi/6,
    ],
])

for i in range(tt):
    env.robots.set_joint_position_targets(finger_pos * i / tt) # 
    world.step(render=True)

# Stage last: pull

In [31]:
graps_pos, grasp_rot

(array([[0.49999976, 0.1903346 , 0.90226483]], dtype=float32),
 array([[0.38268, 0.     , 0.     , 0.92388]], dtype=float32))

In [32]:
# graps_pos[...,0] -= 0.05

In [33]:
for i in range(200):
    graps_pos[...,0] -= 0.002
#   env.robots.set_world_poses(graps_pos, grasp_rot)
    env.xforms.set_world_poses(graps_pos, grasp_rot)
    env.robots.set_joint_position_targets(finger_pos)
    world.step(render=True)

In [2]:
world.reset()

NameError: name 'world' is not defined